In [1]:
import csv
from datetime import datetime
from functions import *
import json
import multiprocessing
import pandas as pd
import sys
import time
import tweepy
from tweepy.auth import OAuthHandler

configuration = json.load(open("configuration.json"))

# Pull a user's friends from the Twitter API

In [19]:
def pull_friends(api, user_ids):
    friendships = []
    error_accounts = []
    
    for user_id in user_ids:
        friend_ids = []
        collected_at = datetime.now()

        try:
            for page in tweepy.Cursor(api.friends_ids, id=user_id).pages():
                friend_ids.extend(page)
            
            friendships.extend([(user_id, friend_id, collected_at) for friend_id in friend_ids])

        except tweepy.RateLimitError:
            time.sleep(15 * 60)
            user_friendships, error_ids = pull_friends(api, [user_id])
            friendships.extend(user_friendships)
            error_accounts.extend(error_ids)
        
        except tweepy.TweepError as ex:
            error_accounts.append((user_id, ex.response.status_code, ex.response.text))
    
    return (friendships, error_accounts)


# Write out the friendships
Currently written for Postgres

In [ ]:
def write_friendships_to_database(configuration, friendships, start_interval, end_interval):
    config_file = configuration["db_config_file"]
    for line in open(db_config_file).readlines():
        key, value = line.strip().split("=")
        config[key] = value
    
    table_name = config_file["table_prefix"] + "_friendships"
    
    ## Create the table if it doesn't exist
    CREATE_STATEMENT = """CREATE TABLE IF NOT EXISTS {table_name} (
                          sample_id BIGINT,
                          user_id BIGINT,
                          friend_id BIGINT,
                          collected_at TIMESTAMP,
                          start_interval TIMESTAMP,
                          end_interval TIMESTAMP);"""
    
    cursor.execute(CREATE_STATEMENT)
    
    ## Insert the rows
    ## Connect to the database and get a cursor object
    database = psycopg2.connect(**config)
    cursor = database.cursor()

    INSERT_STATEMENT = """INSERT INTO {table_name} (sample_id, user_id, friend_id, collected_at, start_interval, end_interval) 
                          VALUES (%s, %s, %s, %s, %s, %s)""".format(table_name=table_name)
    
    rows = [(f[0], f[0], f[1], f[2], start_interval, end_interval) for f in friendships]
    
    ext.execute_batch(cursor, INSERT_STATEMENT, rows)
    
    database.commit()

In [ ]:
def write_errors_to_database(configuration, errors, start_interval, end_interval):
    config_file = configuration["db_config_file"]
    for line in open(db_config_file).readlines():
        key, value = line.strip().split("=")
        config[key] = value
    
    table_name = config_file["table_prefix"] + "_frienship_errors"
    
    ## Connect to the database and get a cursor object
    database = psycopg2.connect(**config)
    cursor = database.cursor()

    ## Create the table if it doesn't exist
    CREATE_STATEMENT = """CREATE TABLE IF NOT EXISTS {table_name} (
                          sample_id BIGINT,
                          api_error_code INT,
                          api_error_text VARCHAR(200),
                          start_interval TIMESTAMP,
                          end_interval TIMESTAMP);"""
    
    cursor.execute(CREATE_STATEMENT)
    
    ## Insert the rows
    INSERT_STATEMENT = """INSERT INTO {table_name} (sample_id, api_error_code, api_error_text, start_interval, end_interval) 
                          VALUES (%s, %s, %s, %s, %s)""".format(table_name=table_name)
    
    rows = [(e[0], e[1], e[2], start_interval, end_interval) for e in errors]
    
    ext.execute_batch(cursor, INSERT_STATEMENT, rows)
    
    database.commit() 